## AST Project 1
### Collaborators: Arian Andalib, Ashley Stone, Jonathan Kho, Emma Oswald
### Michigan State University
### AST 304

In [1]:
########################################################################
# Team Spectacular Stellars: Arian Andalib, Ashley Stone, Jonathan Kho, Emma Oswald
# AST 304, Fall 2022
# Michigan State University
########################################################################

# The libraries used
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.constants as sc
%matplotlib inline
from ode import *

##### 1. Implement routines that advance the solution to a system of ODE’s by one step h for the forward Euler, second-order Runge-Kutta, and fourth3 AST 304 Group Computational Project  order Runge-Kutta. The partially completed routines are in the file ode.py in your project repository. Once you have the routines written, you will need to test them. The file test_ode.py. This test integrates the simpler system of equations
$$z(t = 0) = {0.0, 1.0}, \\ f(t, z, ω) = {\omega z1, −\omega z0} \\ (13) $$ 
##### for which you can verify that the solution is z(t) = {sin(ωt), cos(ωt)}. Run test_ode.py and compare the output of this test with the file sample_output. They should be close.

In [2]:
# unit test for integration methods
# do not alter this file

# our test problem is to integrate
#   dz/dt = [ w*z[1], -w*z[0] ]
# with initial conditions
#   z(t=0) = [ 0.0, 1.0 ].
# The solution is z = [ sin(w*t), cos(w*t) ].  For this test, we set
# w = 2*pi, and integrate from t = 0 to t = 2.
#

import numpy as np
from ode import fEuler, rk2, rk4

integration_methods = {
        'Euler': fEuler,
        'RK2': rk2,
        'RK4': rk4
        }

def f(t,z,w):
    """
    function returning RHS of our ODE.
    
    Arguments
        t (scalar)
        z (2 dimensional array)            
        w (scalar)
    Returns
        dzdt (2-D numpy array)
    """
    dzdt = np.zeros_like(z)
    dzdt[0] =  w*z[1]
    dzdt[1] = -w*z[0]
    return dzdt

def soln(t,w):
    """
    returns analytical solution of ODE
    Arguments
        t (scalar or array-like)
            independent variable
        w (scalar)
            parameter in system of ODEs
    Returns
        2-d solution array at times in argument t
    """
    return np.array([np.sin(w*t),np.cos(w*t)])

def do_one(method):
    # set initial conditions
    z = np.zeros(2)
    z[1] = 1.0

    # period is 1.0, frequency is 2*pi, stepsize is 1/100 of a period
    P = 1.0
    w = 2.0*np.pi/P
    h = P/100.0

    # we'll integrate from t = 0 to t = t_f = 2*P
    t = 0.0
    t_f = 2*P
    # Number of steps
    N = int(t_f/h)

    # check everything!
    print('integrating from t = {0} to t = {1} with {2} steps; h = {3:5.3f}\n'.\
        format(t,t_f,N,h))
    
    # print every 5th line
    PRINT_INTERVAL = 5
    # counter for steps
    cnt = 0
    # format for outputing results
    fmt = '{0:5d}{1:7.3f}   {2:7.3f}{3:7.3f}{4:9.2e}   {5:7.3f}{6:7.3f}{7:9.2e}'
    # format for header
    head_fmt = '{0:>5s}{1:>7s}   {2:>7s}{3:>7s}{4:>9s}   {5:>7s}{6:>7s}{7:>9s}'
    print(head_fmt.format(
        'step','t','z0','s0','|z0-s0|','z1','s1','|z1-s1|')
    )
    stepper = integration_methods[method]
    for step in range(N):
        z = stepper(f,t,z,h,args=w)
        t += h
        cnt += 1
        if (cnt % PRINT_INTERVAL == 0):
            zs = soln(t,w)
            resid = np.abs(z-zs)
            print(fmt.format(cnt,t,z[0],zs[0],resid[0],z[1],zs[1],resid[1]))

print('\n====================Forward Euler====================')
do_one('Euler')
print('\n================2nd order Runge-Kutta================')
do_one('RK2')
print('\n================4th order Runge-Kutta================')
do_one('RK4')



====================Forward Euler====================
integrating from t = 0.0 to t = 2.0 with 200 steps; h = 0.010

 step      t        z0     s0  |z0-s0|        z1     s1  |z1-s1|
    5  0.050     0.312  0.309 2.66e-03     0.961  0.951 9.54e-03
   10  0.100     0.599  0.588 1.10e-02     0.826  0.809 1.66e-02
   15  0.150     0.833  0.809 2.35e-02     0.606  0.588 1.87e-02
   20  0.200     0.989  0.951 3.77e-02     0.323  0.309 1.41e-02
   25  0.250     1.050  1.000 5.05e-02     0.002 -0.000 2.17e-03
   30  0.300     1.010  0.951 5.87e-02    -0.325 -0.309 1.63e-02
   35  0.350     0.869  0.809 5.96e-02    -0.627 -0.588 3.95e-02
   40  0.400     0.639  0.588 5.11e-02    -0.873 -0.809 6.42e-02
   45  0.450     0.342  0.309 3.25e-02    -1.038 -0.951 8.69e-02
   50  0.500     0.005 -0.000 4.55e-03    -1.104 -1.000 1.04e-01
   55  0.550    -0.340 -0.309 3.06e-02    -1.061 -0.951 1.10e-01
   60  0.600    -0.657 -0.588 6.92e-02    -0.914 -0.809 1.05e-01
   65  0.650    -0.916 -0.809 1.07e-0

##### 2. Next, complete the functions in kepler.py that compute the kinetic, potential, and total energies, all per unit reduced mass, as well as the function
def derivs(t,z,m):
##### which computes dr/dt, dv/dt following eq. [7].

##### 3.  Also in the file kepler.py you will find a routine
 def integrate_orbit(z0,m,tend,h,method=’RK4’):
##### that integrates the equations of motion from 0 < t ≤ tend. Notice that this routine takes an optional parameter method with default value ’RK4’:if you call the routine with method=’Euler’ it will use the ode.f Euler method, and similarly for ’RK2’ and ’RK4’. This allows you to switch between integration methods without having to rewrite code. Finally,there is a routine
def set_initial_conditions(a, m, e):
##### that computes the initial position and velocity, as well as the energy and orbital period, given the semi-major axis, mass, and eccentricity.You will need to complete the routines in kepler.py, including the documentation.

##### 4. Write a python script that uses the functions in ode.py and kepler.py to do the following. For each of the three integration methods, integrate the equations of motion over 3 orbital periods, and compute the relative error in the total energy at the end of this time. Take the semi-major axis a = 1 and total mass m = 1. Make the ellipse have an eccentricity of e = 0.5 so that x0 = (1 + e)a = 1.5a. Do each integration for a range of step sizes h =h0, h0/2, h0/4, . . . h0/1024, where h0 = 0.1 T with T being the expected orbital period. Plot the error in the energy as a function of h. Does it scale as expected? Is it better to use linear or logarithmic axes in plotting the error?

##### 5. For the smallest and largest values of h for each of the three integration methods, plot the particle trajectory. Does the orbit close? Is it an ellipse? Does it have the correct semi-major axis? Also plot the energies— potential, kinetic, and total—as a function of time. Put the energies all on the same plot.
